In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import modules as mod

### Global parameters

In [2]:
# Number of market participants 
N = 1600 

# Number of fundamentalists and chartists
N_FUND = 800
N_CHART = 800

# Initial price of the asset traded
P_0 = 100

# Global fundamental price 
P_F = 120

# Variation around the global fundamental price 
THETA = 30

# Maximum extension for chartists' timesteps window 
T_MAX = 15

# Sensitivity of forecasts for fundamentalists
PHI = 2.0

# Sensitivity of forecasts for chartists 
KAPPA = 2.0

# level of information dissipation among the agents
ALPHA = 0.95

# Weight of the market imbalance
DELTA = 0.05

# Intervals of variation for the stochastic noise term 
SIGMA = 30

# Sensitivity threshold for choosing the status
TAO = 20

# Asset quantity for each trader
Q = 50

# Information Threshhold
Ith = 1

# Initializing traders' wealth (W), money (M), and asset quantity(Q)
M = 35000  
Q = 50 
W = [M + Q * P_0 for _ in range(N)] 

In [3]:
# Function to calculate expected price for one fundamentalist
def ep_fundamentalist(p_t, P_F):
    """
    Calculate the expected price for a fundamentalist trader

    Parameters:
    p_t (float): The current market price of the asset
    p_f (float): The perceived fundamental price of the asset for the trader

    Returns:
    float: The expected price for the asset for a fundamentalist trader
    """
    noise = np.random.uniform(-SIGMA, SIGMA)
    # Was not too sure about using an uniform or normal distribution here
    p_f = np.random.uniform(P_F - THETA, P_F + THETA)
    return p_t + PHI * (p_f - p_t) + noise

# Function to calculate expected price for one chartist
def ep_chartist(p_t, past_prices):
    """
    Calculate the expected price for a chartist trader

    Parameters:
    p_t (float): The current market price of the asset
    past_prices (list of float): A list of past market prices of the asset

    Returns:
    float: The expected price for the asset as calculated by a chartist trader
    """
    assert len(past_prices) == T_MAX, "past_prices must be a list of 15 values"
    chartist_T = np.random.randint(2, T_MAX)
    p_T = np.mean(past_prices[-chartist_T:]) 
    noise = np.random.uniform(-SIGMA, SIGMA)
    return p_t + KAPPA/(chartist_T) * (p_t - p_T) + noise

# Function that determines the choice one random trader makes
def rand_trader():
    choice = np.random.randint(0, 3)
    return choice
    

### 

In [61]:
# Function to calculate the order type for a non-random trader
def determine_order_type(expected_price, p_t, TAO, money, asset_quantity):
    """
    Determine the order type of a trader based on the expected price, current price,
    sensitivity threshold, available money, and asset quantity

    Parameters:
    expected_price (float): The expected price of the asset
    p_t (float): The current market price of the asset
    TAO (int): The threshold to decide whether to hold
    money (float): The amount of money the trader has
    asset_quantity (int): The quantity of the asset the trader holds

    Returns:
    str: The order type ('buy', 'sell', or 'hold')
    """

    if abs(expected_price - p_t) < TAO:
        return 'hold' + str(abs(expected_price - p_t))
    elif expected_price > p_t and money > 0:
        return 'buy'
    elif expected_price < p_t and asset_quantity > 0:
        return 'sell'
    else:
        return 'hold'

def set_order_price(order_type, expected_price, current_price, money):
    """
    Set the order price for a trader based on their order type, expected price,
    current price, and the amount of money they have

    Parameters:
    order_type (str): The type of order ('buy', 'sell', or 'hold')
    expected_price (float): The expected price of the asset
    current_price (float): The current market price of the asset
    money (float): The amount of money the trader has
    
    Returns:
    float: The price set for the order
    """
    if order_type == 'buy':
        return np.random.uniform(0, min(money, expected_price))
    elif order_type == 'sell':
        return np.random.uniform(expected_price, current_price)
    else:
        return 0

In [62]:
# Example usage
current_market_price = P_0
trader_money = M
trader_asset_quantity = Q

ep = ep_fundamentalist(current_market_price, P_F) 
order_type = determine_order_type(ep, P_0, TAO, trader_money, trader_asset_quantity)
order_price = set_order_price(order_type, ep, current_market_price, trader_money)

print(ep)
print(order_type)
print(order_price)

131.4161221455049
buy
36.96932465692435


In [63]:
# Initialize agent attributes at T = 0

# structure of agent data
# agents = np.zeros(N, dtype=[('type', 'U10'),# 0
#                             ('wealth', 'f8'),# 1
#                             ('money', 'f8'), # 2
#                             ('assets', 'f8'), # 3
#                             ('expected_price', 'f8'), # 4
#                             ('decision', 'U10'), # 5
#                             ('info', 'f8')]) # 6

agents_dict = {}
past_price = np.repeat(P_0, T_MAX)
for i in range(N_FUND):
    ep = ep_fundamentalist(current_market_price, P_F) 
    
    agents_dict[i] = ('fund',
                 W[i],
                 M,
                 Q,
                 ep,
                 determine_order_type(ep, P_0, TAO, trader_money, trader_asset_quantity),
                 np.random.uniform(0, Ith)
                )

# Start from where N_FUND stops for integers
for i in range(N_FUND, N_FUND+N_CHART):
    ep = ep_chartist(current_market_price, past_price)
    agents_dict[i] = ('chart',
                 W[i],
                 M,
                 Q,
                 ep,
                 determine_order_type(ep, P_0, TAO, trader_money, trader_asset_quantity),
                 np.random.uniform(0, Ith)
                )


In [64]:
class OrderBook:
    def __init__(self, delta, agents_dict):
        """
        Initialize the OrderBook with the given delta value.

        Parameters:
        - delta (float): Parameter for price adjustment.
        """
        self.bids = []  # List to store bid orders
        self.asks = []  # List to store ask orders
        self.delta = delta  # Parameter for price adjustment
        self.N_a = 0
        self.N_b = 0
        self.N_T = 0
        self.agents_dict = agents_dict
    
        
    def place_bid(self, trader_id, price):
        """
        Place a bid order in the order book.

        Parameters:
        - trader_id (int): ID of the trader placing the bid.
        - price (float): Bid price.
        """
        self.bids.append({'trader_id': trader_id, 'price': price})
        self.N_b += 1  # Increment the number of bidders
        
    def place_ask(self, trader_id, price):
        """
        Place an ask order in the order book.

        Parameters:
        - trader_id (int): ID of the trader placing the ask.
        - price (float): Ask price.
        """
        self.asks.append({'trader_id': trader_id, 'price': price})
        self.N_a += 1  # Increment the number of askers
        


    def match_orders(self):
        """
        Match bid and ask orders in the order book.

        Returns:
        - list: List of dictionaries representing matched transactions.
        """
        self.bids.sort(key=lambda x: x['price'], reverse=True)
        self.asks.sort(key=lambda x: x['price'])
        N_T = 0 
        while self.bids and self.asks:
            best_bid = self.bids[0]
            best_ask = self.asks[0]

            if best_bid['price'] >= best_ask['price']:
                # Remove matched orders
                p_L = self.asks[0]['price']
                self.agents_dict[self.asks[0]['trader_id']][3] -= 1 ## assets
                self.agents_dict[self.bids[0]['trader_id']][3] += 1 
                self.agents_dict[self.asks[0]['trader_id']][2] += p_L  
                self.agents_dict[self.bids[0]['trader_id']][2] -= p_L
                self.bids.pop(0)
                self.asks.pop(0)
                self.N_b -= 1
                self.N_a -= 1
                N_T += 1
            
            else:
                break  # No more possible transactions

        return N_T, p_L

    def set_aggregate_price(self, prev_price, N_a, N_b):
        """
        Set the aggregate asset price at time t + 1 based on different cases.

        Parameters:
        - prev_price (float): Previous global asset price at time t.
        - N_a (int): Number of askers in the order book.
        - N_b (int): Number of bidders in the order book.

        Returns:
        - float: New global asset price at time t + 1.
        """
        print('PREV', prev_price)

        if N_a == 0 and N_b > 0:
            new_price = prev_price + self.delta * N_b
            print('1')
        elif N_a > 0 and N_b == 0:
            new_price = prev_price - self.delta * N_a
            print('2')
        elif 0 < N_a < N_b and self.bids[0]['price'] < self.asks[0]['price']:
            new_price = prev_price + self.delta * N_b
            print('3')
        elif 0 < N_a < N_b and self.bids[0]['price'] > self.asks[0]['price']:
            N_T,p_L = self.match_orders()
            new_price = p_L + self.delta * (N_b - N_T)
            print('4')
        elif 0 < N_b < N_a and self.bids[0]['price'] <= self.asks[0]['price']:
            new_price = prev_price - self.delta * N_a
            print('5')
        elif 0 < N_b < N_a and self.bids[0]['price'] > self.asks[0]['price']:
            N_T, p_L = self.match_orders()
            new_price = p_L - self.delta * (N_a - N_T)
            print('6')
        else:
            new_price = prev_price
            print('7')
        print('NEW', new_price)
        return new_price


### Simulation

In [65]:
# Initialize agent attributes at T = 0

# structure of agent data
# agents = np.zeros(N, dtype=[('type', 'U10'),# 0
#                             ('wealth', 'f8'),# 1
#                             ('money', 'f8'), # 2
#                             ('assets', 'f8'), # 3
#                             ('expected_price', 'f8'), # 4
#                             ('decision', 'U10'), # 5
#                             ('info', 'f8')]) # 6

agents_dict = {}
past_price = [P_0 for i in range(T_MAX)]
for i in range(N_FUND):
    ep = ep_fundamentalist(P_0, P_F) 
    agents_dict[i] = ['fund',
                 W[i],
                 M,
                 Q,
                 ep,
                 determine_order_type(ep, P_0, TAO, M, Q),
                 np.random.uniform(0, Ith)
                     ]

# # Start from where N_FUND stops for integers
# for i in range(N_FUND, N_FUND+N_CHART):
#     ep = ep_chartist(P_0, past_price)
#     agents_dict[i] = ['chart',
#                  W[i],
#                  M,
#                  Q,
#                  ep,
#                  determine_order_type(ep, P_0, TAO, trader_money, trader_asset_quantity),
#                  np.random.uniform(0, Ith)
#                      ]

In [66]:
# Simulation 1 cycle
ob = OrderBook(DELTA, agents_dict)
current_market_price = P_0
simulations = 100
# Check #5, decision in dict to get decision
for sim in range(simulations):
    print(ob.agents_dict[0][4])
    for agent_id in ob.agents_dict:
        decision = ob.agents_dict[agent_id][5]
        order_price = set_order_price(decision, ob.agents_dict[agent_id][4], current_market_price, ob.agents_dict[agent_id][2])
        if decision == "buy":
            ob.place_bid(agent_id, order_price)
        elif decision == "sell":
            ob.place_ask(agent_id, order_price)
    current_market_price = ob.set_aggregate_price(current_market_price,
                                                 ob.N_a,
                                                 ob.N_b)
    past_price.pop()
    past_price.append(current_market_price)
    for agent_id in range(800): 
        ep = ep_fundamentalist(current_market_price, P_F) 
        ob.agents_dict[agent_id][4] = ep
        ob.agents_dict[agent_id][5] = determine_order_type(ep,
                                                    current_market_price, 
                                                    TAO, ob.agents_dict[i][2],
                                                    ob.agents_dict[i][3])
    print(ob.agents_dict[0][4])
#     # Start from where N_FUND stops for integers
#     for i in range(800, 1600):
#         ep = ep_chartist(current_market_price, past_price) 
#         ob.agents_dict[i][4] = ep
#         ob.agents_dict[i][5] = determine_order_type(ep,
#                                                     current_market_price, 
#                                                     TAO, ob.agents_dict[i][2],
#                                                     ob.agents_dict[i][3])
#     sell_count = 0
#     buy_count = 0
#     for agent_id in ob.agents_dict:
#         if ob.agents_dict[agent_id][5] == "buy":
#             buy_count +=1
#         if ob.agents_dict[agent_id][5] == "sell":
#             sell_count +=1
#     print('s',sell_count)
#     print('b',buy_count)

142.37679249644265
PREV 100
3
NEW 127.25
46.87118733638022
46.87118733638022
PREV 127.25
3
NEW 163.65
87.27925133559933
87.27925133559933
PREV 163.65
5
NEW 104.75
146.5269210576284
146.5269210576284
PREV 104.75
5
NEW 42.05
251.66765909491542
251.66765909491542
PREV 42.05
3
NEW 142.4
50.29216270467515
50.29216270467515
PREV 142.4
3
NEW 244.3
17.555363125408185
17.555363125408185
PREV 244.3
5
NEW 113.25
100.23766320094686
100.23766320094686
PREV 113.25
5
NEW -26.55000000000001
261.3919917618263
261.3919917618263
PREV -26.55000000000001
3
NEW 134.3
101.90834898202064
101.90834898202064
PREV 134.3
3
NEW 299.90000000000003
-50.02652508121674
-50.02652508121674
PREV 299.90000000000003
5
NEW 98.20000000000002
110.56086470399809
110.56086470399809
PREV 98.20000000000002
5
NEW -105.25
426.50974574133653
426.50974574133653
PREV -105.25
3
NEW 128.45000000000002
142.55507393091682
142.55507393091682
PREV 128.45000000000002
3
NEW 369.1
-156.96359057738997
-156.96359057738997
PREV 369.1
5
NEW 106.55

In [67]:
ob.agents_dict[0][4] = 98
print(ob.agents_dict[0][4])


98


In [41]:
print(current_market_price)

6381.419300895756


In [11]:
past_price.pop()
past_price.append(current_market_price)
ob.agents_dict # Use this
for agent_id in range(800): 
    ep = ep_fundamentalist(current_market_price, P_F) 
    ob.agents_dict[i][4] = ep
    ob.agents_dict[i][5] = determine_order_type(ep,
                                                current_market_price, 
                                                TAO, ob.agents_dict[i][2],
                                                ob.agents_dict[i][3])

# Start from where N_FUND stops for integers
for i in range(800, 1600):
    ep = ep_chartist(current_market_price, past_price) 
    ob.agents_dict[i][4] = ep
    ob.agents_dict[i][5] = determine_order_type(ep,
                                                current_market_price, 
                                                TAO, ob.agents_dict[i][2],
                                                ob.agents_dict[i][3])

Debug Info: Expected Price: -72.43836894607284, Current Price: 251.46792442817636, TAO: 20, Money: 35000, Asset Quantity: 50
Debug Info: Expected Price: -37.52484230288388, Current Price: 251.46792442817636, TAO: 20, Money: 35000, Asset Quantity: 50
Debug Info: Expected Price: -6.869508702847728, Current Price: 251.46792442817636, TAO: 20, Money: 35000, Asset Quantity: 50
Debug Info: Expected Price: -9.585095166123043, Current Price: 251.46792442817636, TAO: 20, Money: 35000, Asset Quantity: 50
Debug Info: Expected Price: -25.07431356152589, Current Price: 251.46792442817636, TAO: 20, Money: 35000, Asset Quantity: 50
Debug Info: Expected Price: 49.50041823584546, Current Price: 251.46792442817636, TAO: 20, Money: 35000, Asset Quantity: 50
Debug Info: Expected Price: -82.09851612211918, Current Price: 251.46792442817636, TAO: 20, Money: 35000, Asset Quantity: 50
Debug Info: Expected Price: -87.81195918255958, Current Price: 251.46792442817636, TAO: 20, Money: 35000, Asset Quantity: 50
D

Debug Info: Expected Price: 300.7408171956444, Current Price: 251.46792442817636, TAO: 20, Money: 35000, Asset Quantity: 50
Debug Info: Expected Price: 296.14490335686395, Current Price: 251.46792442817636, TAO: 20, Money: 35000, Asset Quantity: 50
Debug Info: Expected Price: 290.9506192515076, Current Price: 251.46792442817636, TAO: 20, Money: 35079.58389418585, Asset Quantity: 49
Debug Info: Expected Price: 258.21440812053015, Current Price: 251.46792442817636, TAO: 20, Money: 34928.02481023351, Asset Quantity: 51
Debug Info: Expected Price: 295.2059904117401, Current Price: 251.46792442817636, TAO: 20, Money: 35072.95302887698, Asset Quantity: 49
Debug Info: Expected Price: 275.427748779888, Current Price: 251.46792442817636, TAO: 20, Money: 35000, Asset Quantity: 50
Debug Info: Expected Price: 303.7698436824692, Current Price: 251.46792442817636, TAO: 20, Money: 35075.580800744254, Asset Quantity: 49
Debug Info: Expected Price: 306.75445196830395, Current Price: 251.46792442817636,

In [12]:
ob.agents_dict[805]

['chart', 40000, 35000, 50, 294.8284295380668, 'buy', 0.7717167097142366]

In [13]:
def determine_order_type(expected_price, p_t, TAO, money, asset_quantity):
    """
    Determine the order type of a trader based on the expected price, current price,
    sensitivity threshold, available money, and asset quantity

    Parameters:
    expected_price (float): The expected price of the asset
    p_t (float): The current market price of the asset
    TAO (int): The threshold to decide whether to hold
    money (float): The amount of money the trader has
    asset_quantity (int): The quantity of the asset the trader holds

    Returns:
    str: The order type ('buy', 'sell', or 'hold')
    """

    if abs(expected_price - p_t) < TAO:
        return 'hold' + str(abs(expected_price - p_t))
    elif expected_price > p_t and money > 0:
        return 'buy'
    elif expected_price < p_t and asset_quantity > 0:
        return 'sell'
    else:
        print('we are here')
        return 'hold'

['chart', 40000, 35000, 50, 296.4461422438476, 'buy', 0.5393441389092514]
['chart', 40000, 35000, 50, 293.8153512448475, 'buy', 0.5763455465715824]
['chart', 40000, 34930.453431570946, 51, 305.74795405284317, 'buy', 0.9343597609004183]
['chart', 40000, 35075.696324623874, 49, 273.83856467343713, 'buy', 0.34764389367794246]
['chart', 40000, 34927.35701282616, 51, 259.3949369896448, 'hold7.927012561468445', 0.6318072972107922]
['chart', 40000, 35000, 50, 294.8284295380668, 'buy', 0.7717167097142366]
['chart', 40000, 35000, 50, 285.7035760950407, 'buy', 0.17754771453736629]
['chart', 40000, 34931.30321488526, 51, 301.81705433580976, 'buy', 0.6912823214249678]
['chart', 40000, 34926.90140884134, 51, 271.5995955058107, 'buy', 0.7890545458592175]
['chart', 40000, 35073.20767017735, 49, 301.94795472942536, 'buy', 0.27325793559107336]
['chart', 40000, 35107.556106024545, 49, 286.6445922718927, 'buy', 0.3241249097875255]
['chart', 40000, 35154.153807291856, 48, 289.7624537463686, 'buy', 0.66838

['chart', 40000, 35000, 50, 306.13158251887353, 'buy', 0.5995462139512775]
['chart', 40000, 35000, 50, 298.0540820940585, 'buy', 0.18900877759519963]
['chart', 40000, 35000, 50, 315.2412995488255, 'buy', 0.6913742190705973]
['chart', 40000, 35000, 50, 326.3489719017756, 'buy', 0.008511430122588703]
['chart', 40000, 35000, 50, 304.3464838221492, 'buy', 0.12061775866987912]
['chart', 40000, 35000, 50, 261.6127142864089, 'hold10.144789858232542', 0.25083355898417437]
['chart', 40000, 35000, 50, 278.4849326456558, 'buy', 0.5090451336505822]
['chart', 40000, 35000, 50, 281.58257584423797, 'buy', 0.2624501294425977]
['chart', 40000, 35000, 50, 284.4263718970471, 'buy', 0.9113554571545011]
['chart', 40000, 35000, 50, 251.40597396272187, 'hold0.0619504654544869', 0.8272794173396129]
['chart', 40000, 35000, 50, 292.4937908832987, 'buy', 0.07108579499655088]
['chart', 40000, 35000, 50, 263.86575916413886, 'hold12.397834735962505', 0.9730322418374624]
['chart', 40000, 35000, 50, 280.8842540798297